# Group Lab 3 – Comparative Urban Change in US States

Authors: Andrew Baker, Jamie Marken, James Lyou, Alberto Melendez, Emmanual Robi

#### Part 1

#### Part 2

In [2]:
#Import pandas and geopandas
import pandas as pd
import geopandas

In [3]:
#Read the SAEP data
fp = "./saep_bg10/saep_bg10.shp"
data = geopandas.read_file(fp)

     STATEFP10 COUNTYFP10 TRACTCE10 BLKGRPCE10       GEOID10     NAMELSAD10  \
0           53        001    950100          1  530019501001  Block Group 1   
1           53        001    950100          2  530019501002  Block Group 2   
2           53        001    950100          3  530019501003  Block Group 3   
3           53        001    950200          1  530019502001  Block Group 1   
4           53        001    950200          2  530019502002  Block Group 2   
5           53        001    950200          3  530019502003  Block Group 3   
6           53        001    950300          1  530019503001  Block Group 1   
7           53        001    950300          2  530019503002  Block Group 2   
8           53        001    950300          3  530019503003  Block Group 3   
9           53        001    950300          4  530019503004  Block Group 4   
10          53        001    950300          5  530019503005  Block Group 5   
11          53        001    950400          1  5300

In [4]:
#Read the FIPS dbf data
fp2 = "./WashingtonFIPS.dbf"
dbf = geopandas.read_file(fp2)

      CountyName FIPSCounty geometry
0          Adams        001     None
1         Asotin        003     None
2         Benton        005     None
3         Chelan        007     None
4        Clallam        009     None
5          Clark        011     None
6       Columbia        013     None
7        Cowlitz        015     None
8        Douglas        017     None
9          Ferry        019     None
10      Franklin        021     None
11      Garfield        023     None
12         Grant        025     None
13  Grays Harbor        027     None
14        Island        029     None
15     Jefferson        031     None
16          King        033     None
17        Kitsap        035     None
18      Kittitas        037     None
19     Klickitat        039     None
20         Lewis        041     None
21       Lincoln        043     None
22         Mason        045     None
23      Okanogan        047     None
24       Pacific        049     None
25  Pend Oreille        051     None
2

In [104]:
#Divides SAEP data into counties and write the individual divisions to JSON files
county_counter = 0
for row in dbf.FIPSCounty:
    is_in_county = data['COUNTYFP10'] == row
    subset = data[is_in_county]
    subset.to_file(dbf.CountyName[county_counter] + '.json', driver = 'GeoJSON')
    county_counter += 1

In [5]:
#Creates a new dataframe with total populations of each county in 2017
#and prints the top 10 most populous counties
county_pop = pd.DataFrame()
county_pop['FIPSCounty'] = dbf.FIPSCounty
county_pop['CountyName'] = dbf.CountyName
county_pop['Population'] = 0
counter = 0
    
for row in county_pop.FIPSCounty:
    is_in_county = data['COUNTYFP10'] == row
    subset = data[is_in_county]
    county_pop.Population[counter] = sum(subset['POP2017'] + county_pop.Population[counter])
    counter += 1

county_pop = county_pop.sort_values('Population', ascending=False)
print(county_pop.head(n=10))

C:\Users\James\Anaconda2\envs\GEOG458-Tut\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


   FIPSCounty CountyName  Population
16        033       King     2153700
26        053     Pierce      859399
30        061  Snohomish      789400
31        063    Spokane      499800
5         011      Clark      470999
33        067   Thurston      276900
17        035     Kitsap      264299
38        077     Yakima      253000
36        073    Whatcom      216299
2         005     Benton      193499
